In [2]:
from pathlib import Path
from PIL import Image
from PIL import ImageDraw, ImageFont
from PIL.ExifTags import TAGS
import re
from pillow_heif import register_heif_opener
from tqdm.auto import tqdm
register_heif_opener()  # Register HEIC support in Pillow

c:\Users\chaoran\Desktop\photo-print-resize\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def find_date(img_path: str, im: Image.Image) -> str | None:
    filename = Path(img_path).name
    search = re.search('202[0-9][01][0-9][0-3][0-9]', filename)
    date_str = None
    if search:
        date_str = search.group(0)
        date_str = date_str[:4] + '-' + date_str[4:6] + '-' + date_str[6:8]
    else:
        exif = {}
        exifdata = im.getexif()
        for tag_id in exifdata:
            # get the tag name, instead of human unreadable tag id
            tag = TAGS.get(tag_id, tag_id)
            data = exifdata.get(tag_id)
            exif[tag] = data
        date_str = exif.get('DateTime', None)
        if date_str:
            date_str = date_str[:10].replace(':', '-')
    return date_str


def draw_date(img_path: str, im: Image.Image) -> Image.Image:
    im2 = im.copy()
    w, h = im2.size
    margin = int(0.05 * min(w, h))
    x = margin
    y = h - 1.2*margin
    # Load a TTF font file
    # Use a path to .ttf if not in system path
    font = ImageFont.truetype("arial.ttf", size=32)
    draw = ImageDraw.Draw(im2)
    text = find_date(img_path, im)
    draw.text((x, y), text, font=font, fill="white")
    return im2


def prepare_3R_photo(img_path: str, output_path: str|None = None, overwrite:bool = False) -> Image.Image | None:
    
    img_path:Path = Path(img_path)

    if output_path:
        output_file_path = Path(output_path).joinpath(Path(img_path).stem + '.jpg')
        if output_file_path.exists() and (not overwrite):
            return None

    im = Image.open(img_path)
    # Step 1: Crop to 10:7 or 7:10 based on orientation

    def center_crop_to_aspect(im, aspect_w=10, aspect_h=7):
        width, height = im.size
        im_aspect = width / height
        target_aspect = aspect_w / aspect_h

        if im_aspect > 1:  # landscape
            target_aspect = 10 / 7
        else:
            target_aspect = 7 / 10

        if im_aspect > target_aspect:
            new_width = int(height * target_aspect)
            new_height = height
        else:
            new_width = width
            new_height = int(width / target_aspect)

        left = (width - new_width) // 2
        top = (height - new_height) // 2
        return im.crop((left, top, left + new_width, top + new_height))

    im = center_crop_to_aspect(im)

    # Step 2: Resize to 3R dimensions (at 300 DPI)

    long = 1500
    short = int(long / 10 * 7)

    if im.width < im.height:
        im = im.resize((short, long), Image.LANCZOS)
    else:
        im = im.resize((long, short), Image.LANCZOS)

    im = draw_date(img_path, im)

    if output_path:
        output_file_path.parent.mkdir(exist_ok=True, parents=True)
        if im.mode == "RGBA":
            im = im.convert("RGB")
        im.save(output_file_path, format='JPEG')

    return im

In [16]:
input_dir = r"C:\Users\chaoran\Downloads\ZiAn4"
jpg = sorted(Path(input_dir).glob("*.jpg"))
heic = sorted(Path(input_dir).glob("*.heic"))
png = sorted(Path(input_dir).glob("*.png"))
for img_path in tqdm(jpg + png + heic, desc = 'converting 3R photos', unit = 'image'):
    _ = prepare_3R_photo(img_path, output_path='./output', overwrite=False)

converting 3R photos: 100%|██████████| 309/309 [00:38<00:00,  7.95image/s]
